In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
from datetime import datetime
import sqlite3
from sqlite3 import Error
import pandas as pd
from geopy import distance
import math
import time
import utilities as ut

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
DB = "/Users/funsooje/Documents/GTL/DB/gtlc3.db"
conn = sqlite3.connect(DB)
earthRadius = 6371 # this is in KM

In [5]:
# Functions
l_e_upperLat = 0
l_e_lowerLat = 0
l_e_upperLon = 0
l_e_lowerLon = 0
l_e_id = 0
l_e_level = 0


def linkGrid(lon, lat, locid, approx = True):
#check if grids are empty
    c = conn.cursor()
    c.execute("""select exists (select 1 from locationGrids)""")
    cc = c.fetchall()
    cc = cc[0][0]
    if cc == 0:
        gridId = createNewGrid(lon, lat)
        # return saveLocGrid(locid, gridId)

    # find exact match first
    gridId = findExactGrid(lon, lat)
    
    if gridId == 0:
        if approx: 
            # create the new grid from self
            gridId = createNewGrid(lon, lat)
        else: 
            # if no match, find closest and calculate exact match
            closestCenLon, closestCenLat, _ = findClosestGrid(lon, lat)
            # resolve the lon
            nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat)

            while not (nULon >= lon and nLLon <= lon):
                brng = 90 if lon > closestCenLon else -90
                closestCenLon = round(tuple(distance.distance(meters=50).destination((closestCenLat, closestCenLon), bearing=brng))[1], 7)
                nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat)

            while not (nULat >= lat and nLLat <= lat):
                brng = 0 if lat > closestCenLat else 180
                closestCenLat = round(tuple(distance.distance(meters=50).destination((closestCenLat, closestCenLon), bearing=brng))[0], 7)
                nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat)

            gridId = createNewGrid(closestCenLon, closestCenLat)
    # create new location grid
    if gridId > 0:
        return saveLocGrid(locid, gridId)



def saveLocGrid(locid, gridId):
    c = conn.cursor()
    sql = "insert into locationGrids(locationId, gridId) VALUES(" + str(locid) + "," + str(gridId) + ")"
    c.execute(sql)
    conn.commit()
    return c.lastrowid



def getNewGridParams(lon, lat, level = 1):
    # get limits
    mDis = (math.pow(2, level-1) * 25)  # 2 meter overlap
    # print(lon, lat)
    upperLat = round(tuple(distance.distance(meters=mDis).destination((lat, lon), bearing=0))[0], 7)
    lowerLat = round(tuple(distance.distance(meters=mDis).destination((lat, lon), bearing=180))[0], 7)
    upperLon = round(tuple(distance.distance(meters=mDis).destination((lat, lon), bearing=90))[1], 7)
    lowerLon = round(tuple(distance.distance(meters=mDis).destination((lat, lon), bearing=-90))[1], 7)
    return upperLon, lowerLon, upperLat, lowerLat


def createNewGrid(lon, lat, level = 1):
    centerLat = lat
    centerLon = lon

    # get limits
    upperLon, lowerLon, upperLat, lowerLat = getNewGridParams(lon, lat, level)

    # save to database
    c = conn.cursor()
    #save to db
    sql = """ insert into grids(centerLon, centerLat, upperLon, upperLat, lowerLon, lowerLat, level) VALUES(?,?,?,?,?,?,?)  """
    vals = (centerLon, centerLat, upperLon, upperLat, lowerLon, lowerLat, level)
    c.execute(sql, vals)
    conn.commit()
    return c.lastrowid


def findExactGrid(lon, lat, level = 1):
    global l_e_id
    global l_e_upperLat
    global l_e_lowerLat
    global l_e_upperLon
    global l_e_lowerLon
    global l_e_level
    
    if l_e_id > 0:
        if l_e_upperLat >= lat and l_e_lowerLat <= lat and l_e_upperLon >= lon and l_e_lowerLon <= lon and level == l_e_level:
            return l_e_id
    c = conn.cursor()
    c.execute("""select id, upperLat, lowerLat, upperLon, lowerLon from grids 
                where upperLat >= ?
                AND lowerLat <= ?
                AND upperLon >= ?
                AND lowerLon <= ?
                AND level = ? 
                LIMIT 1
                """, 
                (lat, lat, lon, lon, level))
    cc = pd.DataFrame(c.fetchall())
    if len(cc) > 0:
        l_e_id = cc[0][0]
        l_e_upperLat = cc[1][0]
        l_e_lowerLat = cc[2][0]
        l_e_upperLon = cc[3][0]
        l_e_lowerLon = cc[4][0]
        l_e_level = level
        return l_e_id
    else:
        l_e_id = 0
        return l_e_id
   

def findClosestGrid(lon, lat, level = 1):
    c = conn.cursor()

    c.execute("""SELECT
                    centerLon, centerLat,
                    ABS(centerLat - ?) + ABS(centerLon - ?) AS closest
                FROM
                    grids
                WHERE
                    level = ?
                ORDER BY
                    closest
                LIMIT 1
                """, 
                (lat, lon, level))
    cc = pd.DataFrame(c.fetchall())
    if len(cc) > 0:
        return cc[0][0], cc[1][0], True
    else:
        return 0, 0, False


def mapParent(gridID, lon, lat, level, approx = True):
    # find exact grid of parent
    parentLevel = level + 1
    parentGridId = findExactGrid(lon, lat, parentLevel)
    # match - update grid
    if parentGridId > 0: 
        updateParent(gridID, parentGridId)
        return
    # no match
    if approx: 
        parentGridId = createNewGrid(lon, lat, parentLevel)
        updateParent(gridID, parentGridId)
        return
    else:
        # - find parent closest grid
        closestCenLon, closestCenLat, isExist = findClosestGrid(lon, lat, parentLevel)
        if mDebug:
            print("Closest Grid: ", end = "")
            print(closestCenLon, closestCenLat, isExist)

        # no match - create first parent grid 
        if not isExist:
            nULon, nLLon, nULat, nLLat = getNewGridParams(lon, lat, parentLevel)
            # create using the edge and not the center
            parentGridId = createNewGrid(nLLon, nLLat, parentLevel)
            updateParent(gridID, parentGridId)
            return
        # match 
        mDis = math.pow(2, parentLevel-1) * 25
        jumpDis = ((mDis * 2)) # 2 meter overlap

        # find distance between closest and current on lat
        
        mC = 0
        nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat, parentLevel)
        while not (nULat >= lat and nLLat <= lat):
            closestCenter = (closestCenLat, closestCenLon)
            poI = (lat, closestCenLon)
            disccc = distance.distance(closestCenter, poI).meters
            jumps = int(disccc / jumpDis)
            jumps = 1 if jumps == 0 else jumps
            mC += 1
            brng = 0 if lat > closestCenLat else 180
            closestCenLat = round(tuple(distance.distance(meters=(jumpDis * jumps)).destination((closestCenLat, closestCenLon), bearing=brng))[0], 7)
            nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat)
            if mDebug:
                print(mC, jumps, nULon, nLLon, nULat, nLLat, closestCenLat, brng)
            # print(closestCenLon, closestCenLat)
            # print(nULon, nLLon, nULat, nLLat)
            if mC > 100:
                break

        # find distance between closest and current on lon
        # check a few times
        mC = 0    
        nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat, parentLevel)
        while not (nULon >= lon and nLLon <= lon):
            closestCenter = (closestCenLat, closestCenLon)
            poI = (closestCenLat, lon)
            disccc = distance.distance(closestCenter, poI).meters
            jumps = int(disccc / jumpDis)
            jumps = 1 if jumps == 0 else jumps
            mC += 1
            brng = 90 if lon > closestCenLon else -90
            closestCenLon = round(tuple(distance.distance(meters=jumpDis * jumps).destination((closestCenLat, closestCenLon), bearing=brng))[1], 7)
            nULon, nLLon, nULat, nLLat = getNewGridParams(closestCenLon, closestCenLat, parentLevel)
            if mDebug:
                print(mC, jumps, nULon, nLLon, nULat, nLLat, closestCenLon, brng)
            if mC > 100:
                break 
        
        if mDebug:
            print("upper lon", nULon - lon)
            print("lower lon", nLLon - lon)
            print("upper lat", nULat - lat)
            print("lower lat", nLLat - lat)

        # create the new grid from the new center
        if nULat >= lat and nLLat <= lat and nULon >= lon and nLLon <= lon:
            parentGridId = createNewGrid(closestCenLon, closestCenLat, parentLevel)
            updateParent(gridID, parentGridId)
        else:
            # update to bad grid to be revisited
            updateParent(gridID, -1)
        return
    

def updateParent(gridID, parentGridId):
    c = conn.cursor()
    c.execute("update grids set parentID = " + str(parentGridId) + " where id = " + str(gridID))
    conn.commit()
    return


In [6]:
%%script false --no-raise-error
# old method
# get no parent grids and process parents
mDebug = False

def updateParentGrids(limit = 1):
    conn = None
    try:
        conn = sqlite3.connect(DB)
        c = conn.cursor()
        c.execute("""SELECT
        id,
        centerLon,
        centerLat,
        level
    FROM
        grids
    WHERE
        level < 15
        AND parentID IS NULL
    ORDER BY 
        level DESC, centerLat, centerLon
        """)
        noParentGrids = c.fetchall()

        noParentGrids = pd.DataFrame(noParentGrids)
        if len(noParentGrids) < limit:
            return 0
            
        print(str(len(noParentGrids)), "no parent grids found")

        for index, row in noParentGrids.iterrows():
            gridID = row[0]
            lon = row[1]
            lat = row[2]
            level = int(row[3])
            if mDebug:
                print(gridID, lat, lon, level)
            mapParent(gridID, lon, lat, level)
            if index % 1000 == 0:
                print(str(index), " of ", str(len(noParentGrids)), " - ", str(round(index / len(noParentGrids) * 100, 2)),
                            "%")
        #     if index > 5000:
        #         break # for
        #     break # for
        # break # while
        return len(noParentGrids)
    except Error as e:
        sys.exit(e)
    finally:
        if conn:
            conn.close()


In [7]:
%%script false --no-raise-error
# old method
# get walking locations without grids
cc = 0
while True:
    conn = None
    try:
        conn = sqlite3.connect(DB)
        c = conn.cursor()
        cquery = """
        SELECT
            loc.id AS location_id,
            loc.longitude AS location_longitude,
            loc.latitude AS location_latitude
        FROM
            locations loc
            LEFT JOIN locationGrids lg ON lg.locationId = loc.id
            WHERE lg.locationId IS NULL
            ORDER BY location_longitude, location_latitude
        LIMIT 10000
        """
        # cquery = "SELECT acs.location_id, acs.location_longitude, acs.location_latitude " 
        # cquery = cquery + " FROM v_loc_acs acs LEFT JOIN locationGrids lg ON lg.locationId = acs.location_id "
        # cquery = cquery + " WHERE lg.locationId IS NULL "
        # # cquery = cquery + " WHERE acs.activity = '" + run_act + "' AND lg.locationId IS NULL "
        # cquery = cquery + " ORDER BY acs.location_longitude, acs.location_latitude LIMIT 10000"
        c.execute(cquery)
        noGridLocs = c.fetchall()

        noGridLocs = pd.DataFrame(noGridLocs)
        print(str(len(noGridLocs)), "no grid locations found")

    except Error as e:
        sys.exit(e)
    finally:
        if conn:
            conn.close()

    # run through and build level 1 grids
    lenNoGrids = len(noGridLocs)
    if lenNoGrids == 0:
        break

    start = time.process_time()
    msgBreak = 10000
    
    for index, row in noGridLocs.iterrows():
        locID = row[0]
        locLon = row[1]
        locLat = row[2]

        linkGrid(locLon, locLat, locID)
            
        # if index % (msgBreak/10) == 0:
        #     tt = round(time.process_time() - start, 2)
        #     eta = round((lenNoGrids - index) / msgBreak * tt / 60, 2)
        #     print("{} of {} - {}% - {} secs - est: {} mins".format(index, 
        #                                                             lenNoGrids, 
        #                                                             round(index / lenNoGrids * 100, 2),
        #                                                             tt,
        #                                                             eta))
        #     updateParentGrids(5000)
        #     start = time.process_time()
    tt = round(time.process_time() - start, 2)
    cc += 10000
    print("{} of {} - {}% - {} - {} secs".format(index, lenNoGrids, round(index / lenNoGrids * 100, 2), cc, tt))
    # updateParentGrids()
    


In [8]:
%%script false --no-raise-error
# old method
# run until no more levels
while True:
    if updateParentGrids() == 0:
        break
    


[ ] Get the batch

[ ] Get min and max on lat and lon to get eligible parent grid

[ ] run through batch and stack found matches

[ ] run through batch and stack new parents

[ ] push found batches and parents in one go


In [9]:
# approximate batched functions
def getEligibleParents(minLon, maxLon, minLat, maxLat, level = 1, debug = False):
    c = conn.cursor()
    cquery = """
    select id, upperLat, lowerLat, upperLon, lowerLon from grids 
                    where upperLat <= {}
                    AND lowerLat >= {}
                    AND upperLon <= {}
                    AND lowerLon >= {}
                    AND level = {}
    """.format(maxLat, minLat, maxLon, minLon, level)
    # c.execute("""select id, upperLat, lowerLat, upperLon, lowerLon from grids 
    #                 where upperLat <= ?
    #                 AND lowerLat >= ?
    #                 AND upperLon <= ?
    #                 AND lowerLon >= ?
    #                 AND level = ? 
    #                 """, 
    #                 (maxLat, minLat, maxLon, minLon, level))
    if debug:
        print(cquery)
    c.execute(cquery)
    cc = pd.DataFrame(c.fetchall()).rename(columns = {0: 'id', 1: 'upperLat', 2: 'lowerLat', 3: 'upperLon', 4: 'lowerLon'})
    return cc

def getParent(row, eligibleParents):
    lat = row['location_latitude']
    lon = row['location_longitude']
    if len(eligibleParents) == 0:
        return 0
    id = eligibleParents[(eligibleParents['upperLat'] >= lat) & 
                    (eligibleParents['lowerLat'] <= lat) & 
                    (eligibleParents['lowerLon'] <= lon) & 
                    (eligibleParents['lowerLon'] <= lon)]
    if (len(id) == 0):
        return 0
    else:
        return id.iat[0, 0]

def createUnmmapedParent(row, unmappedParentsDF, level = 1):
    lat = row['location_latitude']
    lon = row['location_longitude']
    centerLat = lat
    centerLon = lon

    # check unmappedParentsDF first
    if len(unmappedParentsDF) > 0:
        id = unmappedParentsDF[(unmappedParentsDF['upperLat'] >= lat) & 
                    (unmappedParentsDF['lowerLat'] <= lat) & 
                    (unmappedParentsDF['lowerLon'] <= lon) & 
                    (unmappedParentsDF['lowerLon'] <= lon)]
        if len(id) > 0:
            return None # it's already mapped - nothing to do

    # else: 
    # get limits
    upperLon, lowerLon, upperLat, lowerLat = getNewGridParams(lon, lat, level)

    a = {'centerLon': centerLon, 'centerLat': centerLat, 'upperLon': upperLon, 'upperLat': upperLat, 'lowerLon': lowerLon, 'lowerLat': lowerLat, 'level': level}
    return a

def padMinMax(minLat, minLon, maxLat, maxLon, level):
    if level >= 14:
        minLat -= 10
        minLon -= 10
        maxLat += 10
        maxLon += 10
    elif level >= 10:
        minLat -= 1
        minLon -= 1
        maxLat += 1
        maxLon += 1
    elif level >= 6:
        minLat -= 0.1
        minLon -= 0.1
        maxLat += 0.1
        maxLon += 0.1 
    else:
        minLat -= 0.01
        minLon -= 0.01
        maxLat += 0.01
        maxLon += 0.01
    return minLat, minLon, maxLat, maxLon


In [10]:
def runlocbatch(n = 10000):
    # Get the batch
    c = conn.cursor()
    cquery = """
    SELECT
        id AS location_id,
        longitude AS location_longitude,
        latitude AS location_latitude
    FROM
        locations
    WHERE gridId IS NULL
    ORDER BY location_longitude, location_latitude
    LIMIT {}
    """.format(n)
    c.execute(cquery)
    noGridLocs = c.fetchall()

    noGridLocs = pd.DataFrame(noGridLocs).rename(columns = {0: 'location_id', 1: 'location_longitude', 2: 'location_latitude'})
    
    if len(noGridLocs) == 0:
        return 0

    print(str(len(noGridLocs)), "no grid locations found")

    # get the unique no grid locs
    uniqueNoGridLocs = noGridLocs.drop_duplicates(subset=['location_longitude', 'location_latitude']).drop('location_id', axis = 1)

    print("{} unique locations found".format(len(uniqueNoGridLocs)))

    # get the min and max lon and lat
    minLon = min(uniqueNoGridLocs['location_longitude'])
    minLat = min(uniqueNoGridLocs['location_latitude'])
    maxLon = max(uniqueNoGridLocs['location_longitude'])
    maxLat = max(uniqueNoGridLocs['location_latitude'])

    minLat, minLon, maxLat, maxLon = padMinMax(minLat, minLon, maxLat, maxLon, 1)

    # get eligible level 1 parents
    l1Eligible = getEligibleParents(minLon, maxLon, minLat, maxLat)

    # get the ones that already have parents
    uniqueNoGridLocs['parentID'] = uniqueNoGridLocs.apply(getParent, args = (l1Eligible, ), axis = 1)

    # map to full list
    uniqueNoGridLocswithParents = uniqueNoGridLocs[uniqueNoGridLocs['parentID'] > 0]
    print("{:,} unique mapped locations found".format(len(uniqueNoGridLocswithParents)))

    mappedLocs = noGridLocs.merge(uniqueNoGridLocswithParents, how = 'inner').drop(columns = ['location_longitude', 'location_latitude']).rename(columns = {'location_id': 'locationId', 'parentID': 'gridId'})

    print("{:,} mapped locations found".format(len(mappedLocs)))

    # push mapped locations to DB
    chunksize = int(len(mappedLocs) / 11)
    n = mappedLocs.to_sql('locationGrids', conn, if_exists='replace', index = False, chunksize = 1000)

    cquery = """
    UPDATE
        locations
    SET
        gridId = lg.gridId
    FROM
        locationGrids lg
    WHERE
        locations.id = lg.locationId
    """
    c.execute(cquery)
    conn.commit()

    unmappedUnique = uniqueNoGridLocs[uniqueNoGridLocs['parentID'] == 0]

    ppcolumns = ['centerLon', 'centerLat', 'upperLon', 'upperLat', 'lowerLon', 'lowerLat', 'level']
    newParentsDF = pd.DataFrame(columns = ppcolumns)

    for _, row in unmappedUnique.iterrows():
        a = createUnmmapedParent(row, newParentsDF)
        if a:
            newParentsDF = pd.concat([newParentsDF, pd.DataFrame(a, index = [0])], ignore_index=True)

    print("{:,} unmmapped unique location - {:,} parents found".format(len(unmappedUnique), len(newParentsDF)))

    # push this to grids
    n = newParentsDF.to_sql('grids', conn, if_exists='append', index = False, chunksize=1000)
    return len(noGridLocs)


In [11]:
def runlevel(level, debug = False):
    c = conn.cursor()
    nextlevel = level + 1

    cquery = """
    SELECT
        id,
        centerLon,
        centerLat,
        level
    FROM
        grids
    WHERE
        level = {}
        AND parentID IS NULL
    ORDER BY 
        centerLat, centerLon
    """.format(level)
    c.execute(cquery)
    lgrids = pd.DataFrame(c.fetchall()).rename(columns = {0: 'id', 1: 'centerLon', 2: 'centerLat', 3: 'level'})

    if len(lgrids) == 0:
        return

    if debug:
        print(lgrids)

    print("{} level {} grids found".format(len(lgrids), level))

    minLon = min(lgrids['centerLon'])
    minLat = min(lgrids['centerLat'])
    maxLon = max(lgrids['centerLon'])
    maxLat = max(lgrids['centerLat'])

    minLat, minLon, maxLat, maxLon = padMinMax(minLat, minLon, maxLat, maxLon, nextlevel)

    if debug:
        print('minLat', 'minLon', 'maxLat', 'maxLon')
        print(minLat, minLon, maxLat, maxLon)

    lgrids['location_latitude'] = lgrids['centerLat']
    lgrids['location_longitude'] = lgrids['centerLon']

    # get eligible next level parents
    l1Eligible = getEligibleParents(minLon, maxLon, minLat, maxLat, nextlevel, debug)
    print("{} level {} parents found".format(len(l1Eligible), nextlevel))

    if debug: 
        print("Eligible Parents")
        print(l1Eligible)

    # get the ones that already have parents
    lgrids['parentID'] = lgrids.apply(getParent, args = (l1Eligible, ), axis = 1)

    # map to full list
    lgridsWithParents = lgrids[lgrids['parentID'] > 0].drop(columns = ['centerLon', 'centerLat', 'level', 'location_latitude', 'location_longitude'])
    print("\t{} grids with parents found".format(len(lgridsWithParents)))
    if debug:
        print(lgridsWithParents)

    # update grids with the following (id with parentID)
    lgridsWithParents.columns = ['gid', 'gParentID']
    if debug:
        print("grid with parents")
        print(lgridsWithParents)
    else:
        lgridsWithParents.to_sql('temp', conn, if_exists='replace', index = False, chunksize=1000)

        cquery = """
        UPDATE
            grids
        SET
            parentID = g2.gParentID
        FROM
            temp g2
        WHERE
            id = g2.gid
        """
        c.execute(cquery)
        conn.commit()

    # without parents
    lgridsNoParent = lgrids[lgrids['parentID'] == 0]

    ppcolumns = ['centerLon', 'centerLat', 'upperLon', 'upperLat', 'lowerLon', 'lowerLat', 'level']
    newParentsDF = pd.DataFrame(columns = ppcolumns)

    for _, row in lgridsNoParent.iterrows():
        a = createUnmmapedParent(row, newParentsDF, nextlevel)
        if a:
            newParentsDF = pd.concat([newParentsDF, pd.DataFrame(a, index = [0])], ignore_index=True)

    print("\t{} grids with no parents - {} unique parents found".format(len(lgridsNoParent), len(newParentsDF)))

    # push this to grids
    if debug:
        print("new Parents")
        print(newParentsDF)
    else:
        n = newParentsDF.to_sql('grids', conn, if_exists='append', index = False, chunksize=1000)


In [12]:
cc = 0
while True:
    c = runlocbatch(1_000_000)
    cc += c
    print("\t{:,} locations processed".format(cc))
    if c == 0:
        break

1000000 no grid locations found
682221 unique locations found
664,127 unique mapped locations found
961,906 mapped locations found
18,094 unmmapped unique location - 3,381 parents found
	1,000,000 locations processed
1000000 no grid locations found
703805 unique locations found
703,704 unique mapped locations found
999,870 mapped locations found
101 unmmapped unique location - 52 parents found
	2,000,000 locations processed
1000000 no grid locations found
728172 unique locations found
712,212 unique mapped locations found
983,086 mapped locations found
15,960 unmmapped unique location - 1,394 parents found
	3,000,000 locations processed
1000000 no grid locations found
604406 unique locations found
598,061 unique mapped locations found
993,015 mapped locations found
6,345 unmmapped unique location - 1,176 parents found
	4,000,000 locations processed
1000000 no grid locations found
483935 unique locations found
476,096 unique mapped locations found
990,719 mapped locations found
7,839 un

In [13]:
for level in range(1, 15):
    print("Level: {}".format(level))
    runlevel(level)
    runlevel(level)
    print()

Level: 1
40091 level 1 grids found
930337 level 2 parents found
	26040 grids with parents found
	14051 grids with no parents - 11910 unique parents found
14051 level 1 grids found
942247 level 2 parents found
	14051 grids with parents found
	0 grids with no parents - 0 unique parents found

Level: 2
11910 level 2 grids found
553347 level 3 parents found
	2710 grids with parents found
	9200 grids with no parents - 7171 unique parents found
9200 level 2 grids found
560381 level 3 parents found
	9200 grids with parents found
	0 grids with no parents - 0 unique parents found

Level: 3
7171 level 3 grids found
315895 level 4 parents found
	1621 grids with parents found
	5550 grids with no parents - 4127 unique parents found
5550 level 3 grids found
319979 level 4 parents found
	5550 grids with parents found
	0 grids with no parents - 0 unique parents found

Level: 4
4127 level 4 grids found
172733 level 5 parents found
	955 grids with parents found
	3172 grids with no parents - 2135 unique 

In [24]:
runlevel(14, True)

## Level Updates

l1y5_o
l2y5_o
l3y_o
l9h

In [5]:
def getNewG1toG9():
    query = """
    SELECT * from v_g1tog9
    """
    g1tog9 = pd.read_sql(query, conn)
    return g1tog9

def getl9h():
    query = """
    SELECT gridId as l9, temporal from l9h
    """
    l9h = pd.read_sql(query, conn)
    return l9h

def getl3y_o():
    query = """
    SELECT gridId as l3, temporal from l3y_o
    """
    l3y_o = pd.read_sql(query, conn)
    return l3y_o

def getl2y5_o():
    query = """
    SELECT gridId as l2, temporal from l2y5_o
    """
    l2y5_o = pd.read_sql(query, conn)
    return l2y5_o

def getl1y5_o():
    query = """
    SELECT gridId as l1, temporal from l1y5_o
    """
    l1y5_o = pd.read_sql(query, conn)
    return l1y5_o

In [6]:
def l9hprocess(locations):
    
    clwl9 = locations[["l9", "temporal"]]
    clwl9['temporal'] = pd.to_datetime(clwl9['temporal'])
    clwl9 = clwl9.drop_duplicates()

    print("{} l9 unique matches".format(len(clwl9)))

    l9h = getl9h()
    l9h['temporal'] = pd.to_datetime(l9h['temporal'])
    clwl9 = clwl9.merge(l9h, how = 'left', indicator=True)
    clwl9 = clwl9[clwl9['_merge'] == 'left_only']
    clwl9 = clwl9[["l9", "temporal"]]

    print("{} l9h matches left".format(len(clwl9)))
    clwl9.rename(columns = {'l9': 'gridId'}, inplace=True)
    clwl9.to_sql('l9h', conn, if_exists='append', index = False, chunksize=1000)
    # print(clwl9.head())

def l3y_o_process(locations):
    clwl = locations[["l3", "temporal"]]
    clwl['datetime'] = clwl['temporal']
    clwl['temporal'] = clwl.apply(ut.yearsnap, axis = 1)
    clwl['temporal'] = pd.to_datetime(clwl['temporal'])
    
    clwl = clwl[['l3', 'temporal']]
    clwl = clwl.drop_duplicates()

    print("{} l3y unique matches".format(len(clwl)))

    l3y_o = getl3y_o()
    l3y_o['temporal'] = pd.to_datetime(l3y_o['temporal'])
    clwl = clwl.merge(l3y_o, how = 'left', indicator=True)
    clwl = clwl[clwl['_merge'] == 'left_only']
    clwl = clwl[["l3", "temporal"]]

    print("{} l3y matches left".format(len(clwl)))
    clwl.rename(columns = {'l3': 'gridId'}, inplace=True)
    clwl.to_sql('l3y_o', conn, if_exists='append', index = False, chunksize=1000)
    # print(clwl.head())

def l2y5_o_process(locations):
    clwl = locations[["l2", "temporal"]]
    clwl['datetime'] = clwl['temporal']
    clwl['temporal'] = clwl.apply(ut.year5snap, axis = 1)
    clwl['temporal'] = pd.to_datetime(clwl['temporal'])
    
    clwl = clwl[['l2', 'temporal']]
    clwl = clwl.drop_duplicates()

    print("{} l2y5 unique matches".format(len(clwl)))

    existing = getl2y5_o()
    existing['temporal'] = pd.to_datetime(existing['temporal'])
    clwl = clwl.merge(existing, how = 'left', indicator=True)
    clwl = clwl[clwl['_merge'] == 'left_only']
    clwl = clwl[["l2", "temporal"]]

    print("{} l2y5 matches left".format(len(clwl)))
    clwl.rename(columns = {'l2': 'gridId'}, inplace=True)
    clwl.to_sql('l2y5_o', conn, if_exists='append', index = False, chunksize=1000)
    # print(clwl.head())

def l1y5_o_process(locations):
    clwl = locations[["l1", "temporal"]]
    clwl['datetime'] = clwl['temporal']
    clwl['temporal'] = clwl.apply(ut.year5snap, axis = 1)
    clwl['temporal'] = pd.to_datetime(clwl['temporal'])
    
    clwl = clwl[['l1', 'temporal']]
    clwl = clwl.drop_duplicates()

    print("{} l1y5 unique matches".format(len(clwl)))

    existing = getl1y5_o()
    existing['temporal'] = pd.to_datetime(existing['temporal'])
    clwl = clwl.merge(existing, how = 'left', indicator=True)
    clwl = clwl[clwl['_merge'] == 'left_only']
    clwl = clwl[["l1", "temporal"]]

    print("{} l1y5 matches left".format(len(clwl)))
    clwl.rename(columns = {'l1': 'gridId'}, inplace=True)
    clwl.to_sql('l1y5_o', conn, if_exists='append', index = False, chunksize=1000)
    # print(clwl.head())

In [7]:
g1tog9 = getNewG1toG9()

In [8]:
for client_id in range(1, 376):
# for client_id in range(330, 358):
	
	print("Client: {}".format(client_id))
	query = """
	SELECT
		gridId AS l1,
		strftime ('%Y-%m-%d %H:00:00',
			datetime) AS temporal
	FROM
		locations
	WHERE
		client_id = {}
	""".format(client_id)
	cl = pd.read_sql(query, conn)
	print("{} locations found".format(len(cl)))
	cl.drop_duplicates(inplace=True)
	print("{} unique matches".format(len(cl)))

	clwl = cl.merge(g1tog9, how='left')
	if len(clwl) == 0:
		continue

	# print("l9h")
	# l9hprocess(clwl)
	print("l3y_o")
	l3y_o_process(clwl)
	# print("l2y5_o")
	# l2y5_o_process(clwl)
	# print("l1y5_o")
	# l1y5_o_process(clwl)

	# break


Client: 1
123968 locations found
19193 unique matches
l3y_o
3497 l3y unique matches
3497 l3y matches left
Client: 2
1185016 locations found
190445 unique matches
l3y_o
29532 l3y unique matches
29420 l3y matches left
Client: 3
2158747 locations found
293557 unique matches
l3y_o
39201 l3y unique matches
38015 l3y matches left
Client: 4
1668966 locations found
354135 unique matches
l3y_o
63381 l3y unique matches
55020 l3y matches left
Client: 5
1058614 locations found
192163 unique matches
l3y_o
13377 l3y unique matches
6839 l3y matches left
Client: 6
162550 locations found
26533 unique matches
l3y_o
4062 l3y unique matches
3253 l3y matches left
Client: 7
1265873 locations found
333736 unique matches
l3y_o
32868 l3y unique matches
17727 l3y matches left
Client: 8
1177083 locations found
281748 unique matches
l3y_o
44614 l3y unique matches
34236 l3y matches left
Client: 9
24100 locations found
5463 unique matches
l3y_o
1283 l3y unique matches
532 l3y matches left
Client: 10
1629 locations 